# NLP model creation and training

In [ ]:
from fastai.gen_doc.nbdoc import *
from fastai.text import * 
from fastai import *

The main thing here is [`RNNLearner`](/text.learner.html#RNNLearner). There are also some utility functions to help create and update text models.

## Quickly get a learner

In [ ]:
show_doc(language_model_learner)

<h4 id="language_model_learner"><code>language_model_learner</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/learner.py#L125" class="source_link">[source]</a></h4>

> <code>language_model_learner</code>(`data`:[`DataBunch`](/basic_data.html#DataBunch), `bptt`:`int`=`70`, `emb_sz`:`int`=`400`, `nh`:`int`=`1150`, `nl`:`int`=`3`, `pad_token`:`int`=`1`, `drop_mult`:`float`=`1.0`, `tie_weights`:`bool`=`True`, `bias`:`bool`=`True`, `qrnn`:`bool`=`False`, `pretrained_model`=`None`, `pretrained_fnames`:`OptStrTuple`=`None`, `kwargs`) → `LanguageLearner`

Create a [`Learner`](/basic_train.html#Learner) with a language model from `data`.  

`bptt` (for backprop trough time) is the number of words we will store the gradient for, and use for the optimization step. 

The model used is an [AWD-LSTM](https://arxiv.org/abs/1708.02182) that is built with embeddings of size `emb_sz`, a hidden size of `nh`, and `nl` layers (the `vocab_size` is inferred from the [`data`](/text.data.html#text.data)). All the dropouts are put to values that we found worked pretty well and you can control their strength by adjusting `drop_mult`. If <code>qrnn</code> is True, the model uses [QRNN cells](https://arxiv.org/abs/1611.01576) instead of LSTMs. The flag `tied_weights` control if we should use the same weights for the encoder and the decoder, the flag `bias` controls if the last linear layer (the decoder) has bias or not.

You can specify `pretrained_model` if you want to use the weights of a pretrained model. If you have your own set of weights and the corrsesponding dictionary, you can pass them in `pretrained_fnames`. This should be a list of the name of the weight file and the name of the corresponding dictionary. The dictionary is needed because the function will internally convert the embeddings of the pretrained models to match the dictionary of the [`data`](/text.data.html#text.data) passed (a word may have a different id for the pretrained model). Those two files should be in the models directory of `data.path`. 

In [ ]:
path = untar_data(URLs.IMDB_SAMPLE)
data = TextLMDataBunch.from_csv(path, 'texts.csv')
learn = language_model_learner(data, pretrained_model=URLs.WT103, drop_mult=0.5)

In [ ]:
show_doc(text_classifier_learner)

<h4 id="text_classifier_learner"><code>text_classifier_learner</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/learner.py#L145" class="source_link">[source]</a></h4>

> <code>text_classifier_learner</code>(`data`:[`DataBunch`](/basic_data.html#DataBunch), `bptt`:`int`=`70`, `emb_sz`:`int`=`400`, `nh`:`int`=`1150`, `nl`:`int`=`3`, `pad_token`:`int`=`1`, `drop_mult`:`float`=`1.0`, `qrnn`:`bool`=`False`, `max_len`:`int`=`1400`, `lin_ftrs`:`Collection`\[`int`\]=`None`, `ps`:`Collection`\[`float`\]=`None`, `kwargs`) → `TextClassifierLearner`

Create a RNN classifier from `data`.  

`bptt` (for backprop trough time) is the number of words we will store the gradient for, and use for the optimization step. 

The model used is the encoder of an [AWD-LSTM](https://arxiv.org/abs/1708.02182) that is built with embeddings of size `emb_sz`, a hidden size of `nh`, and `nl` layers (the `vocab_size` is inferred from the [`data`](/text.data.html#text.data)). All the dropouts are put to values that we found worked pretty well and you can control their strength by adjusting `drop_mult`. If <code>qrnn</code> is True, the model uses [QRNN cells](https://arxiv.org/abs/1611.01576) instead of LSTMs.

The input texts are fed into that model by bunch of `bptt` and only the last `max_len` activations are considerated. This gives us the backbone of our model. The head then consists of:
- a layer that concatenates the final outputs of the RNN with the maximum and average of all the intermediate outputs (on the sequence length dimension),
- blocks of ([`nn.BatchNorm1d`](https://pytorch.org/docs/stable/nn.html#torch.nn.BatchNorm1d), [`nn.Dropout`](https://pytorch.org/docs/stable/nn.html#torch.nn.Dropout), [`nn.Linear`](https://pytorch.org/docs/stable/nn.html#torch.nn.Linear), [`nn.ReLU`](https://pytorch.org/docs/stable/nn.html#torch.nn.ReLU)) layers.

The blocks are defined by the `lin_ftrs` and `drops` arguments. Specifically, the first block will have a number of inputs inferred from the backbone arch and the last one will have a number of outputs equal to data.c (which contains the number of classes of the data) and the intermediate blocks have a number of inputs/outputs determined by `lin_ftrs` (of course a block has a number of inputs equal to the number of outputs of the previous block). The dropouts all have a the same value ps if you pass a float, or the corresponding values if you pass a list. Default is to have an intermediate hidden size of 50 (which makes two blocks model_activation -> 50 -> n_classes) with a dropout of 0.1.

In [ ]:
path = untar_data(URLs.IMDB_SAMPLE)
data = TextClasDataBunch.from_csv(path, 'texts.csv')
learn = text_classifier_learner(data, drop_mult=0.5)

In [ ]:
show_doc(RNNLearner)

<h2 id="RNNLearner"><code>class</code> <code>RNNLearner</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/learner.py#L47" class="source_link">[source]</a></h2>

> <code>RNNLearner</code>(`data`:[`DataBunch`](/basic_data.html#DataBunch), `model`:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module), `bptt`:`int`=`70`, `split_func`:`OptSplitFunc`=`None`, `clip`:`float`=`None`, `adjust`:`bool`=`False`, `alpha`:`float`=`2.0`, `beta`:`float`=`1.0`, `metrics`=`None`, `kwargs`) :: [`Learner`](/basic_train.html#Learner)

Basic class for a [`Learner`](/basic_train.html#Learner) in NLP.  

Handles the whole creation from <code>data</code> and a `model` with a text data using a certain `bptt`. The `split_func` is used to properly split the model in different groups for gradual unfreezing and differential learning rates. Gradient clipping of `clip` is optionally applied. `adjust`, `alpha` and `beta` are all passed to create an instance of [`RNNTrainer`](/callbacks.rnn.html#RNNTrainer). Can be used for a language model or an RNN classifier. It also handles the conversion of weights from a pretrained model as well as saving or loading the encoder.

In [ ]:
show_doc(RNNLearner.get_preds)

<h4 id="RNNLearner.get_preds"><code>get_preds</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/learner.py#L75" class="source_link">[source]</a></h4>

> <code>get_preds</code>(`ds_type`:[`DatasetType`](/basic_data.html#DatasetType)=`<DatasetType.Valid: 2>`, `with_loss`:`bool`=`False`, `n_batch`:`Optional`\[`int`\]=`None`, `pbar`:`Union`\[`MasterBar`, `ProgressBar`, `NoneType`\]=`None`, `ordered`:`bool`=`False`) → `List`\[`Tensor`\]

Return predictions and targets on the valid, train, or test set, depending on `ds_type`.  

If `ordered=True`, returns the predictions in the order of the dataset, otherwise they will be ordered by the sampler (from the longest text to the shortest). The other arguments are passed [`Learner.get_preds`](/basic_train.html#Learner.get_preds).

### Loading and saving

In [ ]:
show_doc(RNNLearner.load_encoder)

<h4 id="RNNLearner.load_encoder"><code>load_encoder</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/learner.py#L61" class="source_link">[source]</a></h4>

> <code>load_encoder</code>(`name`:`str`)

Load the encoder `name` from the model directory.  

In [ ]:
show_doc(RNNLearner.save_encoder)

<h4 id="RNNLearner.save_encoder"><code>save_encoder</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/learner.py#L57" class="source_link">[source]</a></h4>

> <code>save_encoder</code>(`name`:`str`)

Save the encoder to `name` inside the model directory.  

In [ ]:
show_doc(RNNLearner.load_pretrained)

<h4 id="RNNLearner.load_pretrained"><code>load_pretrained</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/learner.py#L66" class="source_link">[source]</a></h4>

> <code>load_pretrained</code>(`wgts_fname`:`str`, `itos_fname`:`str`)

Load a pretrained model and adapts it to the data vocabulary.  

Opens the weights in the `wgts_fname` of `self.model_dir` and the dictionary in `itos_fname` then adapts the pretrained weights to the vocabulary of the <code>data</code>. The two files should be in the models directory of the `learner.path`.

## Utility functions

In [ ]:
show_doc(lm_split)

<h4 id="lm_split"><code>lm_split</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/learner.py#L34" class="source_link">[source]</a></h4>

> <code>lm_split</code>(`model`:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)) → `List`\[[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)\]

Split a RNN `model` in groups for differential learning rates.  

In [ ]:
show_doc(rnn_classifier_split)

<h4 id="rnn_classifier_split"><code>rnn_classifier_split</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/learner.py#L40" class="source_link">[source]</a></h4>

> <code>rnn_classifier_split</code>(`model`:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)) → `List`\[[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)\]

Split a RNN `model` in groups for differential learning rates.  

In [ ]:
show_doc(convert_weights)

<h4 id="convert_weights"><code>convert_weights</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/learner.py#L18" class="source_link">[source]</a></h4>

> <code>convert_weights</code>(`wgts`:`Weights`, `stoi_wgts`:`Dict`\[`str`, `int`\], `itos_new`:`StrList`) → `Weights`

Convert the model `wgts` to go with a new vocabulary.  

Uses the dictionary `stoi_wgts` (mapping of word to id) of the weights to map them to a new dictionary `itos_new` (mapping id to word).

## Get predictions

In [ ]:
show_doc(LanguageLearner, title_level=3)

<h3 id="LanguageLearner"><code>class</code> <code>LanguageLearner</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/learner.py#L87" class="source_link">[source]</a></h3>

> <code>LanguageLearner</code>(`data`:[`DataBunch`](/basic_data.html#DataBunch), `model`:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module), `bptt`:`int`=`70`, `split_func`:`OptSplitFunc`=`None`, `clip`:`float`=`None`, `adjust`:`bool`=`False`, `alpha`:`float`=`2.0`, `beta`:`float`=`1.0`, `metrics`=`None`, `kwargs`) :: [`RNNLearner`](/text.learner.html#RNNLearner)

Subclass of RNNLearner for predictions.  

In [ ]:
show_doc(LanguageLearner.predict)

<h4 id="LanguageLearner.predict"><code>predict</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/learner.py#L90" class="source_link">[source]</a></h4>

> <code>predict</code>(`text`:`str`, `n_words`:`int`=`1`, `no_unk`:`bool`=`True`, `temperature`:`float`=`1.0`, `min_p`:`float`=`None`)

Return the `n_words` that come after `text`.  

If `no_unk=True` the unknown token is never picked. Words are taken randomly with the distribution of probabilities returned by the model. If `min_p` is not `None`, that value is the minimum probability to be considered in the pool of words. Lowering `temperature` will make the texts less randomized. 

## Undocumented Methods - Methods moved below this line will intentionally be hidden

In [ ]:
show_doc(RNNLearner.get_preds)

<h4 id="RNNLearner.get_preds"><code>get_preds</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/learner.py#L75" class="source_link">[source]</a></h4>

> <code>get_preds</code>(`ds_type`:[`DatasetType`](/basic_data.html#DatasetType)=`<DatasetType.Valid: 2>`, `with_loss`:`bool`=`False`, `n_batch`:`Optional`\[`int`\]=`None`, `pbar`:`Union`\[`MasterBar`, `ProgressBar`, `NoneType`\]=`None`, `ordered`:`bool`=`False`) → `List`\[`Tensor`\]

Return predictions and targets on the valid, train, or test set, depending on `ds_type`.  

In [ ]:
show_doc(LanguageLearner.show_results)

<h4 id="LanguageLearner.show_results"><code>show_results</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/learner.py#L105" class="source_link">[source]</a></h4>

> <code>show_results</code>(`ds_type`=`<DatasetType.Valid: 2>`, `rows`:`int`=`5`, `max_len`:`int`=`20`)

Show `rows` result of predictions on `ds_type` dataset.  

## New Methods - Please document or move to the undocumented section